## Parte 1: Recolección de información
#### Web Scraping a "EuropaPress" para investigación de informes referentes a riegos pre electorales

In [1]:
# Importación de librerias
from selenium import webdriver
import undetected_chromedriver as uc
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from random import shuffle
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
import random

# Opciones del navegador
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--headless')  
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument("--disable-notifications")
chrome_options.add_argument("--remote-debugging-port=9222")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
chrome_options.add_experimental_option("excludeSwitches", ["disable-popup-blocking"])

In [2]:
# Clase para personalización
class bcolors:
    WARNING = '\033[93m'
    FAIL = '\033[91m'

# Control de errores
try:   
    # Definición de función
    def Seconds(result):
        hours = round(result/3600)
        minutes = round((result - (hours * 3600))/60)
        seconds = round(result - ((hours * 3600) + (minutes * 60)))
        if seconds < 0: seconds = seconds*-1
        if minutes < 0: minutes = minutes*-1
        final_time = "El sitema tardó: " + str(hours) + " horas " + str(minutes) + " minutos " + str(seconds) + " segundos en finalizar su ejecución"
        return final_time
    
except Exception as e: 
    print(bcolors.FAIL + "Revisa el error => ", e) # Impresión del error personalizado

In [3]:
# Definición de variables
headers = []
titles_array = []
links_array = []
percents_array = []

# Captura del tema de interés de la persona
interest = input("¿Cuál es el riesgo pre electoral de su interés?: ")

# Mensaje para visualización del inicio del proceso
print("Se inició el Web Scraping a 'europa press' del tema: '" + interest + "', por favor espere...")

# Variable para definir el inicio de tiempo en ejecución
start = time.time()

# Acceso y aplicación de filtros en la página
# ****************************************************************************************************************

# Acceso a la página
driver = webdriver.Chrome(
    service = Service(ChromeDriverManager().install()),
    options = chrome_options
)

# Ocultamiento al navegador del uso del driver
driver = uc.Chrome()

driver.get("https://www.europapress.es/buscador.aspx")

# Simulación de comportamiento humano
time.sleep(random.randrange(10,15)) # Toca esta cantidad de tiempo por un captcha

search_bar = driver.find_element(By.CSS_SELECTOR, "input.buscador_CajaTexto")
search_bar.clear()
search_bar.send_keys(interest)

# Simulación de comportamiento humano
time.sleep(random.randrange(3,5))

# Filtro avanzado (botón)
filter_advance_button = driver.find_element(By.XPATH, '//*[@id="BusquedaAvanzada"]')
filter_advance_button.click();

# Simulación de comportamiento humano
time.sleep(random.randrange(3,5))

# Filtro avanzado (fecha)
filter_advance_date = driver.find_element(By.XPATH, '//*[@id="ctl00_CPH_ColumnaIZQ_FechaInicio"]')
filter_advance_date.clear();

# Fecha más antigua de registros (Se hizó verificación manual)
filter_advance_date.send_keys("31/08/1999") 

# Simulación de comportamiento humano
time.sleep(random.randrange(3,5))

# Filtro avanzado (botón)
search_button = driver.find_element(By.XPATH, '//*[@id="Aceptar"]')
search_button.click()

# Captura de noticias
# ****************************************************************************************************************

# Simulación de comportamiento humano
time.sleep(random.randrange(3,5))

# Ciclo para recorrer todas las páginas
while(True):
    
    # Simulación de comportamiento humano
    time.sleep(random.randrange(3,5))
    
    # Div (contiene las noticias o artículos)
    elements = driver.find_elements(By.CSS_SELECTOR, "div.buscador_noticiasFecha")
    
    # Si hay elementos itere si no termine el ciclo y cierre el navegador
    if elements: 
    
        # Ciclo para separa cada uno de los elementos
        for element in elements:

        # Títulos
        # ****************************************************************************************************************
            headers = element.find_elements(By.CSS_SELECTOR, 'h2.buscador_TituloCompletoNoticia')
            if headers: 
                for header in headers:
                    titles_array.append(header.text)
            else: titles_array.append('...')
        # ****************************************************************************************************************

        # Enlaces
        # ****************************************************************************************************************
            links = element.find_elements(By.CSS_SELECTOR, 'a.buscador_TituloNoticia')
            if links: 
                for link in links:
                    links_array.append(link.get_attribute("href"))
            else: links_array.append('...')
        # ****************************************************************************************************************

        # Porcentajes de coincidencia
        # ****************************************************************************************************************
            percents = element.find_elements(By.CSS_SELECTOR, 'progress.buscador_progress')
            if percents: 
                for percent in percents:
                    percents_array.append(percent.text)
            else: percents_array.append('...')
        # ****************************************************************************************************************

        # Construcción del diccionario 
        news = {
            "titles": titles_array,
            "links": links_array,
            "percents": percents_array
        }

        # Construcción inicial del dataframe
        df = pd.DataFrame(news)

        # Modo DEBUG (como el dd de Laravel)
        display(df)

        # Simulación de comportamiento humano
        time.sleep(random.randrange(3,5))

        # Control del error al momento de finalizar el páginado
        try:

            # Simulación de comportamiento humano (clic página siguiente)
            next_button = (driver.find_elements(By.CLASS_NAME, 'buscador_PaginadorSiguiente'))[1]

            # Condicional para validación de existencia del elemento
            if next_button: 
                elements = []
                # Simulación de comportamiento humano
                time.sleep(random.randrange(3,5))
                next_button.click()
            else: 
                break;
                driver.quit()
        except Exception as e:
            display("Exception :" + e)
            driver.quit() 
            break
        except NoSuchElementException as nse:
            display("NoSuchElementException :" + nse)
            driver.quit()
            break
        except TimeoutException as tim:
            display("TimeoutException :" + tim)
            driver.quit()
            break
    else:
        driver.quit()
        break
        
# Cerrar el controlador de Chrome
driver.quit()

¿Cuál es el riesgo pre electoral de su interés?: Manipulación de información en Colombia
Se inició el Web Scraping a 'europa press' del tema: 'Manipulación de información en Colombia', por favor espere...


,titles,links,percents
0,La AN celebra desde mañana el juicio contra 'L...,https://www.europapress.es/nacional/noticia-an...,29%
1,La CNMV multa a Abengoa con 100.000 euros por ...,https://www.europapress.es/andalucia/sevilla-0...,29%
2,"La CNMV multa a Squirrel, Abengoa, Codere y Re...",https://www.europapress.es/economia/noticia-cn...,29%
3,Detienen a cinco personas en Recas y desmantel...,https://www.europapress.es/castilla-lamancha/n...,29%
4,El Constitucional colombiano rechaza el recurs...,https://www.europapress.es/internacional/notic...,29%
5,El Supremo rechaza una denuncia de Uribe contr...,https://www.europapress.es/internacional/notic...,29%
6,El periodista Rafael Vilasanjuan reivindica en...,https://www.europapress.es/catalunya/noticia-p...,29%
7,La Fiscalía de Colombia solicita el sobreseimi...,https://www.europapress.es/internacional/notic...,29%
8,El mundo no aborda las necesidades de salud de...,https://www.infosalus.com/asistencia/noticia-m...,29%
9,La defensa del expresidente Uribe insiste en q...,https://www.europapress.es/internacional/notic...,29%


,titles,links,percents
0,La AN celebra desde mañana el juicio contra 'L...,https://www.europapress.es/nacional/noticia-an...,29%
1,La CNMV multa a Abengoa con 100.000 euros por ...,https://www.europapress.es/andalucia/sevilla-0...,29%
2,"La CNMV multa a Squirrel, Abengoa, Codere y Re...",https://www.europapress.es/economia/noticia-cn...,29%
3,Detienen a cinco personas en Recas y desmantel...,https://www.europapress.es/castilla-lamancha/n...,29%
4,El Constitucional colombiano rechaza el recurs...,https://www.europapress.es/internacional/notic...,29%
5,El Supremo rechaza una denuncia de Uribe contr...,https://www.europapress.es/internacional/notic...,29%
6,El periodista Rafael Vilasanjuan reivindica en...,https://www.europapress.es/catalunya/noticia-p...,29%
7,La Fiscalía de Colombia solicita el sobreseimi...,https://www.europapress.es/internacional/notic...,29%
8,El mundo no aborda las necesidades de salud de...,https://www.infosalus.com/asistencia/noticia-m...,29%
9,La defensa del expresidente Uribe insiste en q...,https://www.europapress.es/internacional/notic...,29%


In [4]:
# Limpieza del DataFrame
df = df.drop_duplicates(subset=["titles"])

In [5]:
# Generación del archivo csv
df.to_csv(interest.replace(" ", "_").lower() + "_EuropaPress.csv")

# Variable para definir el final de tiempo en ejecución
end = time.time()

In [6]:
# Variable para calcular tiempo y para enviar a la función
result = end-start

#Impresión del tiempo que tardó
print(Seconds(result))

El sitema tardó: 0 horas 1 minutos 7 segundos en finalizar su ejeción


In [7]:
df

,titles,links,percents
0,La AN celebra desde mañana el juicio contra 'L...,https://www.europapress.es/nacional/noticia-an...,29%
1,La CNMV multa a Abengoa con 100.000 euros por ...,https://www.europapress.es/andalucia/sevilla-0...,29%
2,"La CNMV multa a Squirrel, Abengoa, Codere y Re...",https://www.europapress.es/economia/noticia-cn...,29%
3,Detienen a cinco personas en Recas y desmantel...,https://www.europapress.es/castilla-lamancha/n...,29%
4,El Constitucional colombiano rechaza el recurs...,https://www.europapress.es/internacional/notic...,29%
5,El Supremo rechaza una denuncia de Uribe contr...,https://www.europapress.es/internacional/notic...,29%
6,El periodista Rafael Vilasanjuan reivindica en...,https://www.europapress.es/catalunya/noticia-p...,29%
7,La Fiscalía de Colombia solicita el sobreseimi...,https://www.europapress.es/internacional/notic...,29%
8,El mundo no aborda las necesidades de salud de...,https://www.infosalus.com/asistencia/noticia-m...,29%
9,La defensa del expresidente Uribe insiste en q...,https://www.europapress.es/internacional/notic...,29%
